<a href="https://colab.research.google.com/github/spatank/CIS-700/blob/master/Project/log_odds_POS_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
os.chdir('/content/Drive/My Drive/CIS-700/')

In [4]:
!ls

Data				  log_odds.ipynb
draco_verb_network_ff.graphml	  log_odds_POS_level.ipynb
global_A_large.mat		  networks.ipynb
global_verb_network_ff.graphml	  __pycache__
harry_verb_network_ff.graphml	  ron_verb_network_ff.graphml
hermione_verb_network_ff.graphml  util.py
hp_narrative_chains.py		  voldemort_verb_network_ff.graphml
log_odds_corpus_level.ipynb


In [5]:
!ls Data

04212020
04292020
Canon
hpcanon_sr_narrative_chains_counts_NNPs.txt
hpc_raw_text.txt
hpff_raw_text_reduced.txt
hpff_raw_text.txt
hpff_sr_narrative_chains_counts_NNPs_new.txt


In [20]:
import collections
import nltk
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
import numpy as np

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
canon_verb_counts = collections.Counter()

with open('Data/hpc_raw_text.txt', encoding = 'utf-8', errors = 'ignore') as f:
  for line in f:
    words = nltk.word_tokenize(line.strip())
    words_no_punctuation = [word.lower() for word in words if word.isalnum()]
    verb_list = []
    for word in words_no_punctuation:
      tag = nltk.tag.pos_tag([word])[0][1]
      if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        verb = lemmatizer.lemmatize(word, 'v')
        verb_list.append(verb)
    canon_verb_counts.update(verb_list)    

In [0]:
# from itertools import islice

# corpus_path = 'Data/hpff_raw_text.txt'
# file = open(corpus_path, encoding = 'UTF-8', errors = 'ignore').readlines()
# file_len = len(file)
# num_lines_keep = round(0.10 * file_len)
# num_lines_discard = file_len - num_lines_keep
# lines_in_sets = [num_lines_keep, num_lines_discard]
# temp = iter(file) 
# splits = [list(islice(temp, 0, ele)) for ele in lines_in_sets] 
# # split 0 is data to keep
# with open('Data/hpff_raw_text_reduced.txt', 'w') as f:
#   for line in splits[0]:
#     f.write('%s\n' % line)

In [0]:
fan_fiction_verb_counts = collections.Counter()

with open('Data/hpff_raw_text_reduced.txt', encoding = 'utf-8', errors = 'ignore') as f:
  for line in f:
    words = nltk.word_tokenize(line.strip())
    words_no_punctuation = [word.lower() for word in words if word.isalnum()]
    verb_list = []
    for word in words_no_punctuation:
      tag = nltk.tag.pos_tag([word])[0][1]
      if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        verb = lemmatizer.lemmatize(word, 'v')
        verb_list.append(verb)
    fan_fiction_verb_counts.update(verb_list) 

In [29]:
keys_a = set(canon_verb_counts.keys())
keys_b = set(fan_fiction_verb_counts.keys())
intersection = keys_a & keys_b # '&' operator is used for set intersection
print(len(intersection))

3011


What words appear more often in fan fiction than the canonical Harry Potter books?

In [30]:
total_num_verbs_in_fan_fiction = sum(fan_fiction_verb_counts.values())
total_num_verbs_in_canon = sum(canon_verb_counts.values())

log_odds_data = {}

for word in canon_verb_counts.keys():
  
  count_canon = canon_verb_counts[word]
  count_fan_fiction = fan_fiction_verb_counts[word]
  
  log_prob_canon = np.log(count_canon) - np.log(total_num_verbs_in_canon)
  log_prob_fan_fiction = np.log(count_fan_fiction) - np.log(total_num_verbs_in_fan_fiction)
  

  if log_prob_fan_fiction != 0:
    log_odds = log_prob_fan_fiction - log_prob_canon
    log_odds_data[word] = log_odds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


In [31]:
log_odds_tuples = [(word, log_odds) for word, log_odds in log_odds_data.items()]
log_odds_tuples = sorted(log_odds_tuples, key=lambda x: x[1], reverse=True)

for i in range(0, 50):
  print(log_odds_tuples[i])

('haired', 4.629232857114514)
('figure', 4.617376243957989)
('date', 3.2007024887195827)
('scold', 3.178896129667528)
('snicker', 3.176247121495951)
('freak', 3.046120309425003)
('remark', 2.947784408704141)
('pen', 2.8787915372171895)
('exclaim', 2.803721564027277)
('base', 2.8008299957474776)
('mock', 2.7452601445926668)
('scrunch', 2.6645369333202265)
('state', 2.626775332136293)
('comment', 2.6037279287246218)
('excuse', 2.5911094647654096)
('pair', 2.586336186012751)
('review', 2.529740518389117)
('radiate', 2.4910260062084255)
('respond', 2.434574993582874)
('pal', 2.431387739888695)
('muse', 2.3972935285457186)
('reply', 2.3878114141208773)
('alfred', 2.3856654905506005)
('blush', 2.2967028934297096)
('realise', 2.2787876883764797)
('pout', 2.237830109827815)
('sniffle', 2.21381523362394)
('school', 2.1927618244261087)
('tease', 2.150510518935299)
('adore', 2.1418417339988522)
('update', 2.1418417339988505)
('rowling', 2.099152325303802)
('stutter', 2.0561862894203564)
('cuddle'